In [ ]:
!pip install pydub

In [ ]:
import kagglehub
path = kagglehub.dataset_download("adrivg/ravdess-emotional-speech-video")
print("Path to dataset files:", path)

100%|██████████| 12.4G/12.4G [09:52<00:00, 22.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from scipy.io import wavfile
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import moviepy.editor as mp
from torchsummary import summary
from torch.nn.functional import one_hot
from torchvision.models import vgg16
from torchvision.models.video import r3d_18

  if event.key is 'enter':



In [ ]:
if torch.cuda.is_available():
    print("GPU is available")
    print("Device:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is available
Device: Tesla T4


In [ ]:
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}
severity_map = {
    1: "Low Intensity",
    2: "High Intensity"
}

In [ ]:
emotion_lst = [(int(x)-1) for x in emotion_map.keys()]
severit_lst = [(int(x)-1) for x in severity_map.keys()]
emotion_map_oh = one_hot(torch.tensor(emotion_lst), num_classes=8)
severity_map_oh = one_hot(torch.tensor(severit_lst), num_classes=2)

In [ ]:
class Mydata(Dataset):
  def __init__(self, rootdir, num_frame = 16, size = (112, 112), sr = 22050, transform = None):
    super().__init__()
    self.rootdir = rootdir
    self.num_frame = num_frame
    self.size = size
    self.sr = sr
    self.transform = transform
    self.filepaths = []

    for (root,dirs,files) in os.walk(self.rootdir):
      for file_name in files:
        if file_name.endswith(".mp4"):
          splits = file_name.split("-")
          if int(splits[0]) == 1: # adding that have both audio and video
            self.filepaths.append(os.path.join(root, file_name))

    print(self.filepaths)

  def __len__(self):
    return len(self.filepaths)

  def extract_frames(self, video_path):

    cap = cv2.VideoCapture(video_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    arr = np.linspace(0, length, self.num_frame)
    frames = []

    for i in arr:
      cap.set(cv2.CAP_PROP_POS_FRAMES, i)
      ret, frame = cap.read()

      if ret:
        frame = cv2.resize(frame, (112, 112))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
      else: # if frame is not read successfully, append a blank frame
        frames.append(np.zeros((112, 112, 3), dtype=np.uint8)) # Append blank frame


    cap.release()
    frames = np.array(frames)
    frames = frames.transpose(3, 0, 1, 2)
    # To (num_Frame, C, H, W)

    return torch.tensor(frames, dtype=torch.float32) / 255.

  def generate_spectrogram(self, video_path):
    temp_audio_path = "temp_audio.wav"

    video = VideoFileClip(video_path)
    video.audio.write_audiofile(temp_audio_path)
    sample_rate, samples = wavfile.read(temp_audio_path)

    # Convert stereo to mono
    if samples.ndim == 2:
        samples = samples.mean(axis=1)

    max_val = np.max(np.abs(samples))

    # Normalize and amplify
    if max_val == 0:
        print("Audio appears to be completely silent.")
        normalized = samples
    else:
        normalized = samples / max_val
    amplified = normalized * 50
    amplified = np.clip(amplified, -1.0, 1.0)

    frequencies, times, Sxx = spectrogram(amplified, self.sr)

    spectrogram_array = 10 * np.log10(Sxx + 1e-10)

    if spectrogram_array.ndim > 2:
        spectrogram_array = np.mean(spectrogram_array, axis=0)

    # Pad or trim to target length
    current_length = spectrogram_array.shape[1]
    target_length = 650
    if current_length < target_length:
        # Pad with zeros
        pad_width = target_length - current_length
        spectrogram_array = np.pad(spectrogram_array, pad_width=((0, 0), (0, pad_width)), mode='constant')
    elif current_length > target_length:
        # Trim to target length
        spectrogram_array = spectrogram_array[:, :target_length]

    return torch.tensor(spectrogram_array, dtype=torch.float32).unsqueeze(0)

  def get_lebel(self, video_path):
    video_path = video_path.split("/")[-1]
    emotion = video_path.split("-")[2]
    severity = video_path.split("-")[3]
    emotion_oh = one_hot(torch.tensor([int(emotion)-1]), num_classes=8).float()
    severity_oh = one_hot(torch.tensor([int(severity)-1]), num_classes=2).float()
    return emotion_oh, severity_oh

  def __getitem__(self, idx):
    video_path = self.filepaths[idx]
    frames = self.extract_frames(video_path)
    spectrogram = self.generate_spectrogram(video_path)
    emotion, severity = self.get_lebel(video_path)

    if self.transform:
      frames = self.transform(frames)
      spectrogram = self.transform(spectrogram)

    return frames, spectrogram, emotion, severity

In [ ]:
class MyModel(nn.Module):
  def __init__(self, num_classes_emotion, num_classes_sev):
    super().__init__()
    self.num_classes = num_classes_emotion
    self.num_classes_sev = num_classes_sev

### Fine tuning r3d

    # Tranning only last 2 conv layer of r3d
    vid_c = r3d_18(pretrained=True)
    for name, para in vid_c.named_parameters():
      if "layer4.0" in name:
        para.requires_grad = True
      else:
        para.requires_grad = False
    # Removing Last and only FC layer in r3d
    vid_c.fc = nn.Identity()

    self.vid_c = vid_c
    # Passing Through one FC layer that will be train
    self.vid_d = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512, 128),
        nn.ReLU(),
    )

###

### Fine tuning vgg16
    aud_c = vgg16(pretrained=True)
    aud_c.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    for name, params in aud_c.named_parameters():
      if  "features.26" in name or "features.28" in name or "features.30" in name : # fine-tuning only last 3 conv layers in features
        params.requires_grad = True
      else:
        params.requires_grad = False
    aud_c.features[0].requires_grad = True
    aud_c.classifier = nn.Identity()

    self.aud_c = aud_c

    # Passing Through one FC layer that will be train
    self.aud_d = nn.Sequential(
        nn.Flatten(),
        nn.Linear(25088, 128),
        nn.ReLU(),
    )
###

    self.fc = nn.Sequential(
        nn.Linear(256, 128),
        nn.ReLU()
        )

    # OutPutting 2 output
    self.output_emo = nn.Linear(128, num_classes_emotion)
    self.output_sev = nn.Linear(128, num_classes_sev)

  def forward(self, x_vid, x_aud):
    #videos
    x_vid = self.vid_c(x_vid)
    x_vid = self.vid_d(x_vid)

    #audio
    x_aud = self.aud_c(x_aud)
    x_aud = self.aud_d(x_aud)

    #concat
    x = torch.cat([x_vid, x_aud], dim=1)

    # after
    x = self.fc(x)

    # output
    output_emo = self.output_emo(x)
    output_emo = nn.Softmax(dim=1)(output_emo)
    output_sev = self.output_sev(x)
    output_sev = nn.Softmax(dim=1)(output_sev)

    return output_emo, output_sev

In [ ]:
def train(model, epochs, lr, dataset, loss_fn_emotion, loss_fn_serverity, accumulation_step):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scaler = torch.cuda.amp.GradScaler() # Making object for Mixed precision

    for epoch in range(epochs):
      for batch_idx, batch in enumerate(dataset):
        frames, spectrogram, emotion, severity = batch
        frames = frames.to(device)
        spectrogram = spectrogram.to(device)
        emotion = emotion.to(device)
        severity = severity.to(device)

        # Forward pass
        ### output_emo, output_sev = model(frames, spectrogram) # Noraml
        with torch.cuda.amp.autocast(): # By AMP api
          output_emo, output_sev = model(frames, spectrogram)

          # Calculate loss

          loss_emo = loss_fn_emotion(torch.squeeze(output_emo), torch.squeeze(emotion))
          loss_sev = loss_fn_serverity(torch.squeeze(output_sev), torch.squeeze(severity))

          loss = loss_emo + loss_sev
          loss = loss / accumulation_step

        # Backward pass and optimization after accumulation
        ### loss.backward() # Normal
        scaler.scale(loss).backward() # By AMP api

        if (batch_idx + 1) % accumulation_step == 0:
          ### optimizer.step() # Normal
          # By AMP api
          scaler.step(optimizer)
          scaler.update()

          optimizer.zero_grad()

        print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss.item()}") #Print loss for monitoring
        if batch_idx % 50 == 0:
            torch.cuda.empty_cache()


In [ ]:
dataset = Mydata(rootdir=path)
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

['/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-06-02-01-01-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-04-02-01-02-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-05-02-01-02-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-08-02-02-02-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-08-01-01-01-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speech-video/versions/1/RAVDESS dataset/Video_Speech_Actor_14/Actor_14/01-01-05-02-02-01-14.mp4', '/root/.cache/kagglehub/datasets/adrivg/ravdess-emotional-speec

In [ ]:
model = MyModel(num_classes_emotion=8, num_classes_sev=2)
model.to(device)
loss_fn_emotion = nn.CrossEntropyLoss()
loss_fn_serverity = nn.CrossEntropyLoss()
accumulation_step = 8
train(model, epochs=10, lr=0.001, dataset=dataloader, loss_fn_emotion=loss_fn_emotion, loss_fn_serverity=loss_fn_serverity, accumulation_step= accumulation_step)


  scaler = torch.cuda.amp.GradScaler() # Making object for Mixed precision



MoviePy - Writing audio in temp_audio.wav


  with torch.cuda.amp.autocast(): # By AMP api



MoviePy - Done.
Epoch: 1/10, Loss: 0.346660315990448
MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
Epoch: 1/10, Loss: 0.339398056268692


...

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
Epoch: 1/10, Loss: 0.3234088122844696


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
Epoch: 1/10, Loss: 0.3234088122844696


MoviePy - Writing audio in temp_audio.wav
